In [ ]:
# pip install git+https://github.com/openai/whisper.git 
# !pip install openai==1.35.13 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import clear_output
from phonemizer import phonemize
import random
import pandas as pd
import collections
import numpy as np
from tqdm import tqdm
import glob


textpath = "../../Dataset/CMU-ARCTIC/transciptions.txt"
file1 = open(textpath, 'r')
Lines = file1.readlines()
transcriptions = {line.split(" ")[1]: line.split('"')[1] for line in Lines}

# pip install openai
from openai import OpenAI
api_key = "sk-ANuLBmSVZUkf8I308b17C775Ce464c7dA06dF119E9A29fE3"
api_base = "https://api.ai-gaochao.cn/v1"
client = OpenAI(api_key=api_key, base_url=api_base)
# def get_response(text):
#     completion = client.chat.completions.create(
#       # model="gpt-3.5-turbo-1106",
#       model="gpt-4o-2024-05-13",
#       messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": text}
#       ]
#     )
#     return completion.choices[0].message.content

def gpt_api_no_stream(prompt: str, 
                      # model='gpt-4o-2024-05-13',
                      model='gpt-3.5-turbo-1106',
                      reset_messages: bool = True,
                      response_only: bool = True
                      ):
    
    messages = [{'role': 'user','content': prompt},]

    try:
        completion = client.chat.completions.create(
            model=model,
            messages=messages,
        )
        completion = dict(completion)
        msg = None
        choices = completion.get('choices', None)
        if choices:
            msg = choices[0].message.content
        else:
            msg = completion.message.content
    except Exception as err:
        return (False, f'OpenAI API error: {err}')
    if reset_messages:
        messages.pop(-1)
    else:
        # add text of response to messages
        messages.append({
            'role': choices[0].message.role,
            'content': choices[0].message.content
        })
    if response_only:
        return True, msg
    else:
        return True, messages

def GetLLMPrompt(sentence, language, phonemized=None):
    words = sentence.split()
    if type(phonemized)!=list:
        phonemized = [phonemize(word, language='en-us', backend='espeak', with_stress=True).split()[0] for word in words]
    shfflephonemized = phonemized

    start = f"""Can you provide me with three {language} words to represent the phoneme sequences delimited by triple backticks. 
For example, in Japanese, "Trail (tɹˈeɪl)" is expected to have Japanese representation of "トレイル"; where "'" in phonemes represents the stress point of the word. 
Here, your task is to provide me with three {language} words that can replace the phoneme senquences, delimited by triple backticks.
Please focus on phonetically similar characters instead of similar characters in terms of the meaning.
The expected output should be in JSON format. 
You can first list three possible choices of the words and then re-order them in order of the similarity of the pronunciation. 
The following is the example in Hindi language.
{{
  "I": {{
    "phonemes": "ˈaɪ",
    "choices": ["आई", "ऐ", "आई"],
    "similarity order": ["आई", "ऐ", "आई"]
  }},
  "love": {{
    "phonemes": "lˈʌv",
    "choices": ["लव", "लव", "लव"],
    "similarity order": ["लव", "लव", "लव"]
  }},
  "you": {{
    "phonemes": "juː",
    "choices": ["यू", "यू", "यू"],
    "similarity order": ["यू", "यू", "यू"]
  }},
}}
```
"""
    for p, ph in enumerate(shfflephonemized):
        start += f"{words[p]}: {ph}\n"
    start = start[:-1]
    start += f"""
```
Again, the responses should be in a JSON format and sort them in order of the similarity to each phoneme sequence.
{{
"""
    for p, ph in enumerate(shfflephonemized):
        start += f"""  "{words[p]}": {{
"phonemes": "{ph}",
"choices": [`1st choices of {language} characters`, `2nd choices of {language} characters`, `3rd choices of {language} characters`],
"similarity order": [`1st most similar {language} characters`, `2nd most similar {language} characters`, `3rd most similar {language} characters`],
}},\n"""
    start = start[:] + "}"
    return start

adds = {
    "zhi": ["the", ["ðɪ"]],
    "za": ["the pineapple", ["ðə", "pˈaɪnæpəl"]],
    "ah": ["a little awkward", ["ɐ","lˈɪɾəl","ˈɔːkwɚd"]],
}

# Evaluate each word
postprocessing = {a: {} for a in adds}
for addname in adds:
# for addname in ["zhi"]:
    sentence, phonemized = adds[addname]
    # for language in ["Hindi", "Korean", "Japanese", "Russian"]:
    for language in ["Hindi", "Korean"]:
    # for language in ["Hindi", "Korean", "Japanese"]:
        filelists = glob.glob(f"./LLM_responses/08-A/{language}/postprocessing_{addname}_*.npy")
        a_list = []
        for path in filelists:
            response = np.load(path).item()
            try:
                a_list += [eval(response[response.index("{"):-1*response[::-1].index("}")]) if response[-1]!="}" else eval(response[response.index("{"):])]
            except ValueError:
                pass
        # print(f"{len(a_list)} / {len(filelists)}")
        # print("Normalized    :", sentence)
        dirs = []
        for a in a_list:
            a = {key: a[key] for key in sentence.split()}
            dirs += [a]
        for i in range(len(dirs)):
            for key in dirs[i]:
                newlist = []
                for j in range(len(dirs[i][key]["similarity order"])):
                    newlist += [dirs[i][key]["similarity order"][j]]*(3-j)
                dirs[i][key]["similarity order"] = newlist
        data = {key: [element for i in range(len(dirs)) for element in dirs[i][key]["similarity order"]] for key in dirs[0]}
        # Get the transliterated sentences
        arrays = []
        counts = []
        for word in sentence.split():
            c = collections.Counter(data[word])
            df = pd.DataFrame(c.items(), columns=["phonemes", "count"]).sort_values("count", ascending=False).values
            arrays += [df[0,0]]
            counts += [list(df[:1,1])]
            
        postprocessing[addname][language] = arrays[0]

import re
from whisper.normalizers.english import EnglishNumberNormalizer, EnglishSpellingNormalizer, remove_symbols_and_diacritics

# keep numbers, -, and '
class EnglishTextNormalizer:
    def __init__(self):
        self.ignore_patterns = r"\b(hmm|mm|mhm|mmm|uh|um)\b"
        self.replacers = {
            # common contractions
            r"\bwon't\b": "will not",
            r"\bcan't\b": "can not",
            r"\blet's\b": "let us",
            r"\bain't\b": "aint",
            r"\by'all\b": "you all",
            r"\bwanna\b": "want to",
            r"\bgotta\b": "got to",
            r"\bgonna\b": "going to",
            r"\bi'ma\b": "i am going to",
            r"\bimma\b": "i am going to",
            r"\bwoulda\b": "would have",
            r"\bcoulda\b": "could have",
            r"\bshoulda\b": "should have",
            r"\bma'am\b": "madam",
            # contractions in titles/prefixes
            r"\bmr\b": "mister ",
            r"\bmrs\b": "missus ",
            r"\bst\b": "saint ",
            r"\bdr\b": "doctor ",
            r"\bprof\b": "professor ",
            r"\bcapt\b": "captain ",
            r"\bgov\b": "governor ",
            r"\bald\b": "alderman ",
            r"\bgen\b": "general ",
            r"\bsen\b": "senator ",
            r"\brep\b": "representative ",
            r"\bpres\b": "president ",
            r"\brev\b": "reverend ",
            r"\bhon\b": "honorable ",
            r"\basst\b": "assistant ",
            r"\bassoc\b": "associate ",
            r"\blt\b": "lieutenant ",
            r"\bcol\b": "colonel ",
            r"\bjr\b": "junior ",
            r"\bsr\b": "senior ",
            r"\besq\b": "esquire ",
            # prefect tenses, ideally it should be any past participles, but it's harder..
            r"'d been\b": " had been",
            r"'s been\b": " has been",
            r"'d gone\b": " had gone",
            r"'s gone\b": " has gone",
            r"'d done\b": " had done",  # "'s done" is ambiguous
            r"'s got\b": " has got",
            # general contractions
            r"n't\b": " not",
            r"'re\b": " are",
            # r"'s\b": " is",
            r"'d\b": " would",
            r"'ll\b": " will",
            r"'t\b": " not",
            r"'ve\b": " have",
            r"'m\b": " am",
        }
        self.standardize_numbers = EnglishNumberNormalizer()
        self.standardize_spellings = EnglishSpellingNormalizer()

    def __call__(self, s: str):
        s = s.lower()

        s = re.sub(r"[<\[][^>\]]*[>\]]", "", s)  # remove words between brackets
        s = re.sub(r"\(([^)]+?)\)", "", s)  # remove words between parenthesis
        s = re.sub(self.ignore_patterns, "", s)
        # s = re.sub(r"\s+'", "'", s)  # when there's a space before an apostrophe

        for pattern, replacement in self.replacers.items():
            s = re.sub(pattern, replacement, s)

        s = re.sub(r"(\d),(\d)", r"\1\2", s)  # remove commas between digits
        s = re.sub(r"\.([^0-9]|$)", r" \1", s)  # remove periods not followed by numbers
        s = remove_symbols_and_diacritics(s, keep=".%$¢€£-'")  # keep numeric symbols

        # s = self.standardize_numbers(s)
        s = self.standardize_spellings(s)

        # now remove prefix/suffix symbols that are not preceded/followed by numbers
        s = re.sub(r"[.$¢€£]([^0-9])", r" \1", s)
        s = re.sub(r"([^0-9])%", r"\1 ", s)

        s = re.sub(r"\s+", " ", s)  # replace any successive whitespaces with a space
        
        return s
normalizer = EnglishTextNormalizer()

def get_original_sentence(basename):
    original_sentence = transcriptions[basename]
    original_sentence = original_sentence.replace(" --", ",")
    return original_sentence

# obrien = []
# for b, basename in enumerate(transcriptions):
#     original_sentence = get_original_sentence(basename)
#     sentence = normalizer(original_sentence)
#     # if len(sentence.split())!=len(original_sentence.split()):
#     candidates = ["I'm", "'ll", "'re", "'s", "'t", "'ve", "'d"]
#     if np.array([key in original_sentence for key in candidates]).sum()==0:
#         if "-" in original_sentence:
#             print(original_sentence)
#             print(sentence)
#             print()

- Get responses

In [ ]:
repeatnum = 3
save = False
fin = []
# for language in ["Hindi", "Korean", "Japanese"]:
for language in ["Korean"]:
    print(language)
    # for key in tqdm(list(transcriptions.keys())):
    for key in ["arctic_a0176"]:
        # if key in allerrors:
        #     pass
        # else:
        #     if len(glob.glob(f"./LLM_responses/08-A/{language}/{key}_*.npy"))>=repeatnum:
        #         continue
        exist_length = len(glob.glob(f"./LLM_responses/08-A/{language}/{key}_*.npy"))
        if exist_length>=repeatnum:
            continue
        else:
            start = exist_length
        osentence = get_original_sentence(key)
        sentence = normalizer(osentence)
        if save:
            prompt = GetLLMPrompt(sentence, language)
        for r in range(start, repeatnum):
            savepath = f"./LLM_responses/08-A/{language}/{key}_{r}.npy"
            if save:
                try:
                    # response = get_response(prompt)
                    response = gpt_api_no_stream(prompt)[1]
                except AuthenticationError:
                    continue
                np.save(savepath, response)
                fin += [key]

- Get the possible sentence from responses

In [ ]:
from IPython.display import clear_output

In [ ]:
# languages = ["Hindi", "Korean"]
# languages = ["Hindi", "Korean"]
languages = ["Hindi", "Korean"]
# languages = ["Hindi", "Korean", "Japanese"]
# languages = ["Hindi", "Korean", "Japanese", "Russian"]
errors = []
added = []
for b, basename in enumerate(list(transcriptions.keys())):
    
    # savepath = f"./LLM_responses/08-A_multi-lingual_text/{basename}.npy"
    savepath = f"./LLM_responses/08-A_multi-lingual_text GPT3-Turbo/{basename}.npy"
    try:
        a = np.load(savepath, allow_pickle=True).item()
        if np.array([la in a for la in languages+["English", "Original English"]]).mean()==1:
            continue
    except FileNotFoundError:
        pass
    # if basename!="arctic_a0016":
    #     continue
    try:
        original_sentence = get_original_sentence(basename)
        sentence = normalizer(original_sentence)

        sentences = {}
        for language in languages:
            rank = 1

            # Evaluate each word
            filelists = []
            filelists += glob.glob(f"./LLM_responses/08-A/{language}/{basename}_*.npy")[:3]
            # filelists += glob.glob(f"./LLM_responses/08-A GPT3-Turbo/{language}/{basename}_*.npy")
            filelists += glob.glob(f"./LLM_responses/08-A GPT3-Turbo/{language}/{basename}_*.npy")[:3]
            assert len(filelists)>=6
            
            a_list = []
            for path in filelists:
                response = np.load(path).item()
                try:
                    a = eval(response[response.index("{"):-1*response[::-1].index("}")]) if response[-1]!="}" else eval(response[response.index("{"):])

                    if len(a)==len(set(sentence.split())):
                        # Check whether the response is valid
                        test = []
                        for word in sentence.split():
                            exist = word in a
                            if not(exist):
                                normalized_word = normalizer.standardize_numbers(word)
                                a_array = np.array(list(a.keys()))
                                bl = normalized_word==a_array
                                exist = bool(bl.sum())
                                if exist:
                                    a[word] = a[normalized_word]
                            ## check the type of data
                            if exist:
                                if type(a[word])!=dict:
                                    exist = False
                            test += [exist]
                        if np.array([test]).mean()==1:
                            a_list += [a]
                except (ValueError, SyntaxError) as e:
                    pass
            dirs = []
            for a in a_list:
                a = {key: a[key] for key in sentence.split()}
                dirs += [a]
            ordernames = []
            for i in range(len(dirs)):
                for key in dirs[i]:
                    newlist = []
                    try:
                        ordername = "similarity order"
                        dirs[i][key][ordername]
                    except KeyError:
                        ordername = "similarity_order"
                    # delete duplicated words
                    candidates = list(set(dirs[i][key][ordername]))
                    newwords = []
                    for tword in dirs[i][key][ordername]:
                        if tword in candidates:
                            candidates.remove(tword)
                            newwords += [tword]
                        if len(candidates)==0:
                            break
                    dirs[i][key][ordername] = newwords
                    for j in range(len(dirs[i][key][ordername])):
                        newlist += [dirs[i][key][ordername][j]]*(len(newwords)-j)
                        # newlist += [dirs[i][key][ordername][j]]
                    dirs[i][key][ordername] = newlist
                ordernames += [ordername]
            data = {key: [element for i in range(len(dirs)) for element in dirs[i][key][ordernames[i]]] for key in dirs[0]}

            # Get the transliterated sentences
            arrays = []
            words = sentence.split()
            for w, word in enumerate(words):
                if word in list(set([a.split(" ")[0] for a in np.array(list(adds.values()))[:, 0]])):
                    if word=="a":
                        arrays += [postprocessing["ah"][language]]
                    if word=="the":
                        pro = phonemize(words[w] + " " + words[w+1], language='en-us', backend='espeak', with_stress=True).split()[0]
                        for the in ["zhi", "za"]:
                            if adds[the][1][0]==pro:
                                break
                        arrays += [postprocessing[the][language]]
                else:
                    c = collections.Counter(data[word])
                    df = pd.DataFrame(c.items(), columns=["phonemes", "count"]).sort_values("count", ascending=False).values
                    arrays += [df[0,0]]

            # put period and comma
            try:
                targets = [".", ","]
                now = 0
                english_arrays = sentence.split()
                for word in original_sentence.split():
                # for word in sentence.split():
                    normalized = normalizer(word)
                    num = len(normalized.split(" "))
                    now += (num-1)
                    for target in targets:
                        if target in word:
                            arrays[now] += target
                            english_arrays[now] += target
                    now += 1
            except IndexError:
                errors += [basename]
                continue

            clear_output(wait=True)
            print(f"{b+1} / {len(transcriptions)}: {basename}")
            print(f"{len(a_list)} / {len(filelists)}")
            print("English       :", original_sentence)
            print("Normalized    :", sentence)
            transliterated = " ".join(arrays)
            print("Transliterated: ", transliterated)

            print()
            sentences["Original English"] = original_sentence
            sentences["English"] = " ".join(english_arrays)
            sentences[language] = transliterated

            for i in range(len(data)):
                c = collections.Counter(data[list(data.keys())[i]])
                print(list(data.keys())[i], "  ", str(c)[8:-1])
    except IndexError:
        added += [basename]
    np.save(savepath, sentences)
    # if b>-1:
        # break

In [ ]:
files = glob.glob("./LLM_responses/08-A_multi-lingual_text/*")
files.sort()

path = files[4]
print(path)
a = np.load(path, allow_pickle=True).item()
for key in a:
    print(a[key])

# VCTK

In [ ]:
num = 0
vctktexts = np.load("VCTK_transcriptions.npy", allow_pickle=True).item()
for key in vctktexts:
    text = vctktexts[key]
    num += len(text.split())

In [ ]:
repeatnum = 3
save = True
fin = []
vctktexts = np.load("VCTK_transcriptions.npy", allow_pickle=True).item()
# for language in ["Hindi", "Korean", "Japanese"]:
for language in ["Hindi"]:
    print(language)
    for key in tqdm(list(vctktexts.keys())[:1500]):
        # exist_length = len(glob.glob(f"./LLM_responses/08-VCTK/{language}/{key}_*.npy"))
        exist_length = len(glob.glob(f"./LLM_responses/08-VCTK GPT3-Turbo/{language}/{key}_*.npy"))
        if exist_length>=repeatnum:
            continue
        else:
            start = exist_length
        osentence = vctktexts[key]
        sentence = normalizer(osentence)
        if save:
            prompt = GetLLMPrompt(sentence, language)
        for r in range(start, repeatnum):
            savepath = f"./LLM_responses/08-VCTK GPT3-Turbo/{language}/{key}_{r}.npy"
            if save:
                try:
                    # response = get_response(prompt)
                    response = gpt_api_no_stream(prompt)[1]
                except AuthenticationError:
                    continue
                np.save(savepath, response)
                fin += [key]

In [ ]:
languages = ["Hindi"]
errors = []
added = []
for b, basename in enumerate(list(vctktexts.keys())[:20]):
    savepath = f"./LLM_responses/08-VCTK_multi-lingual_text/{basename}.npy"
    try:
        a = np.load(savepath, allow_pickle=True).item()
        if np.array([la in a for la in languages+["English", "Original English"]]).mean()==1:
            continue
    except FileNotFoundError:
        pass
    # if basename!="arctic_a0016":
    #     continue
    try:
        original_sentence = vctktexts[basename]
        sentence = normalizer(original_sentence)

        sentences = {}
        for language in languages:
            rank = 1

            # Evaluate each word
            filelists = []
            filelists += glob.glob(f"./LLM_responses/08-VCTK/{language}/{basename}_*.npy")[:3]
            # filelists += glob.glob(f"./LLM_responses/08-VCTK GPT3-Turbo/{language}/{basename}_*.npy")[:3]
            assert len(filelists)>=3
            
            a_list = []
            for path in filelists:
                response = np.load(path).item()
                try:
                    a = eval(response[response.index("{"):-1*response[::-1].index("}")]) if response[-1]!="}" else eval(response[response.index("{"):])

                    if len(a)==len(set(sentence.split())):
                        # Check whether the response is valid
                        test = []
                        for word in sentence.split():
                            exist = word in a
                            if not(exist):
                                normalized_word = normalizer.standardize_numbers(word)
                                a_array = np.array(list(a.keys()))
                                bl = normalized_word==a_array
                                exist = bool(bl.sum())
                                if exist:
                                    a[word] = a[normalized_word]
                            ## check the type of data
                            if exist:
                                if type(a[word])!=dict:
                                    exist = False
                            test += [exist]
                        if np.array([test]).mean()==1:
                            a_list += [a]
                except (ValueError, SyntaxError) as e:
                    pass
            dirs = []
            for a in a_list:
                a = {key: a[key] for key in sentence.split()}
                dirs += [a]
            ordernames = []
            for i in range(len(dirs)):
                for key in dirs[i]:
                    newlist = []
                    try:
                        ordername = "similarity order"
                        dirs[i][key][ordername]
                    except KeyError:
                        ordername = "similarity_order"
                    # delete duplicated words
                    candidates = list(set(dirs[i][key][ordername]))
                    newwords = []
                    for tword in dirs[i][key][ordername]:
                        if tword in candidates:
                            candidates.remove(tword)
                            newwords += [tword]
                        if len(candidates)==0:
                            break
                    dirs[i][key][ordername] = newwords
                    for j in range(len(dirs[i][key][ordername])):
                        newlist += [dirs[i][key][ordername][j]]*(len(newwords)-j)
                        # newlist += [dirs[i][key][ordername][j]]
                    dirs[i][key][ordername] = newlist
                ordernames += [ordername]
            data = {key: [element for i in range(len(dirs)) for element in dirs[i][key][ordernames[i]]] for key in dirs[0]}

            # Get the transliterated sentences
            arrays = []
            words = sentence.split()
            for w, word in enumerate(words):
                if word in list(set([a.split(" ")[0] for a in np.array(list(adds.values()))[:, 0]])):
                    if word=="a":
                        arrays += [postprocessing["ah"][language]]
                    if word=="the":
                        pro = phonemize(words[w] + " " + words[w+1], language='en-us', backend='espeak', with_stress=True).split()[0]
                        for the in ["zhi", "za"]:
                            if adds[the][1][0]==pro:
                                break
                        arrays += [postprocessing[the][language]]
                else:
                    c = collections.Counter(data[word])
                    df = pd.DataFrame(c.items(), columns=["phonemes", "count"]).sort_values("count", ascending=False).values
                    arrays += [df[0,0]]

            # put period and comma
            try:
                targets = [".", ","]
                now = 0
                english_arrays = sentence.split()
                for word in original_sentence.split():
                # for word in sentence.split():
                    normalized = normalizer(word)
                    num = len(normalized.split(" "))
                    now += (num-1)
                    for target in targets:
                        if target in word:
                            arrays[now] += target
                            english_arrays[now] += target
                    now += 1
            except IndexError:
                errors += [basename]
                continue

            clear_output(wait=True)
            print(f"{b+1} / {len(vctktexts)}: {basename}")
            print(f"{len(a_list)} / {len(filelists)}")
            print("English       :", original_sentence)
            print("Normalized    :", sentence)
            transliterated = " ".join(arrays)
            print("Transliterated: ", transliterated)

            print()
            sentences["Original English"] = original_sentence
            sentences["English"] = " ".join(english_arrays)
            sentences[language] = transliterated

            for i in range(len(data)):
                c = collections.Counter(data[list(data.keys())[i]])
                print(list(data.keys())[i], "  ", str(c)[8:-1])
    except IndexError:
        added += [basename]
    np.save(savepath, sentences)
    # if b>-1:
        # break

In [ ]:
files = glob.glob("./LLM_responses/08-VCTK_multi-lingual_text/*")
files.sort()

path = files[4]
print(path)
a = np.load(path, allow_pickle=True).item()
for key in a:
    print(a[key])

# Train, Val, Test Split for ARCTIC

In [ ]:
train, val, test = 932, 100, 100
np.random.seed(0)
randomized = np.random.choice(np.array(list(transcriptions.keys())), 1132, False)

In [ ]:
trainset = list(np.sort(randomized[:train]))
valset = list(np.sort(randomized[train:train+val]))
testset = list(np.sort(randomized[-test:]))

In [ ]:
# np.save("./data_split_ARCTIC.npy", [trainset, valset, testset])
# list(np.load("./data_split_ARCTIC.npy", allow_pickle=True))

In [ ]:
# numbers_error = [
#     'arctic_a0031',
#     'arctic_a0210',
#     'arctic_a0245',
#     'arctic_a0276',
#     'arctic_a0348',
#     'arctic_a0371',
#     'arctic_a0434',
#     'arctic_a0453',
#     'arctic_a0556',
#     'arctic_b0211',
#     'arctic_b0236',
#     'arctic_b0260',
#     'arctic_b0330',
#     'arctic_b0348',
#     'arctic_b0365',
#     'arctic_b0526',
# ]
# obrien_error = [
#     'arctic_a0252',
#     'arctic_a0493',
#     'arctic_a0494',
#     'arctic_a0574',
#     'arctic_b0447',
# ]
# dashsymbol_error =  [
#     'arctic_a0006',
#     'arctic_a0016',
#     'arctic_a0047',
#     'arctic_a0049',
#     'arctic_a0154',
#     'arctic_a0162',
#     'arctic_a0189',
#     'arctic_a0204',
#     'arctic_a0213',
#     'arctic_a0266',
#     'arctic_a0298',
#     'arctic_a0313',
#     'arctic_a0320',
#     'arctic_a0321',
#     'arctic_a0324',
#     'arctic_a0325',
#     'arctic_a0330',
#     'arctic_a0331',
#     'arctic_a0335',
#     'arctic_a0337',
#     'arctic_a0478',
#     'arctic_a0484',
#     'arctic_a0491',
#     'arctic_a0514',
#     'arctic_a0570',
#     'arctic_b0231',
#     'arctic_b0271',
#     'arctic_b0292',
#     'arctic_b0294',
#     'arctic_b0298',
#     'arctic_b0300',
#     'arctic_b0361',
#     'arctic_b0379',
#     'arctic_b0397',
#     'arctic_b0421',
#     'arctic_b0425',
#     'arctic_b0428',
#     'arctic_b0429',
#     'arctic_b0446',
#     'arctic_b0501',
# ]
# including_s_error = [
#     'arctic_a0015',
#     'arctic_a0016',
#     'arctic_a0034',
#     'arctic_a0056',
#     'arctic_a0062',
#     'arctic_a0072',
#     'arctic_a0073',
#     'arctic_a0083',
#     'arctic_a0084',
#     'arctic_a0132',
#     'arctic_a0140',
#     'arctic_a0162',
#     'arctic_a0166',
#     'arctic_a0180',
#     'arctic_a0226',
#     'arctic_a0241',
#     'arctic_a0269',
#     'arctic_a0284',
#     'arctic_a0297',
#     'arctic_a0320',
#     'arctic_a0324',
#     'arctic_a0350',
#     'arctic_a0374',
#     'arctic_a0378',
#     'arctic_a0381',
#     'arctic_a0385',
#     'arctic_a0394',
#     'arctic_a0399',
#     'arctic_a0451',
#     'arctic_a0459',
#     'arctic_a0509',
#     'arctic_a0556',
#     'arctic_a0563',
#     'arctic_a0566',
#     'arctic_a0581',
#     'arctic_a0583',
#     'arctic_a0585',
#     'arctic_a0592',
#     'arctic_b0005',
#     'arctic_b0011',
#     'arctic_b0027',
#     'arctic_b0076',
#     'arctic_b0106',
#     'arctic_b0113',
#     'arctic_b0117',
#     'arctic_b0120',
#     'arctic_b0123',
#     'arctic_b0126',
#     'arctic_b0162',
#     'arctic_b0193',
#     'arctic_b0208',
#     'arctic_b0209',
#     'arctic_b0219',
#     'arctic_b0221',
#     'arctic_b0231',
#     'arctic_b0260',
#     'arctic_b0262',
#     'arctic_b0299',
#     'arctic_b0312',
#     'arctic_b0334',
#     'arctic_b0346',
#     'arctic_b0351',
#     'arctic_b0354',
#     'arctic_b0384',
#     'arctic_b0419',
#     'arctic_b0496',
#     'arctic_b0528',
# ]
# godbless_error = [
#     "arctic_a0006",
# ]
# allerrors = list(set(numbers_error + obrien_error + dashsymbol_error + including_s_error + godbless_error))
# allerrors.sort()

In [ ]:
text = """
Can you provide me with three Japanese words to represent the phoneme sequences delimited by triple backticks. For example, in Japanese, ``Trail (\\textipa{t\\textturnr"eIl})'' is expected to have Japanese representation of ``\\begin{CJK}{UTF8}{min}トレイル''\end{CJK}; where ``''' in phonemes represents the stress point of the word. Here, your task is to provide me with three Japanese words that can replace the phoneme senquences, delimited by triple backticks. Please focus on phonetically similar characters instead of similar characters in terms of the meaning. The expected output should be in JSON format. You can first list three possible choices of the words and then re-order them in order of the similarity of the pronunciation. 
The following is the example in Hindi language.
$[$Few Shot Examples$]$

\\textasciigrave\\textasciigrave\\textasciigrave
Let's: \\textipa{l"Ets} \\
go: \\textipa{g"oU}
\\textasciigrave\\textasciigrave\\textasciigrave
Again, the responses should be in a JSON format and sort them in order of the similarity to each phoneme sequence.
\{
  ``Let's'': \{
``phonemes'': ``\\textipa{l"Ets}'',
``choices'': $[$\\textasciigrave 1st choices of Japanese characters\\textasciigrave , \\textasciigrave 2nd choices of Japanese characters\\textasciigrave , \\textasciigrave 3rd choices of Japanese characters\\textasciigrave $]$,
``similarity order'': $[$\\textasciigrave 1st most similar Japanese characters\\textasciigrave , \\textasciigrave 2nd most similar Japanese characters\\textasciigrave , \\textasciigrave 3rd most similar Japanese characters\\textasciigrave $]$,
\},
  ``go'': \{
``phonemes'': ``\\textipa{g"oU}'',
``choices'': $[$\\textasciigrave 1st choices of Japanese characters\\textasciigrave , \\textasciigrave 2nd choices of Japanese characters\\textasciigrave , \\textasciigrave 3rd choices of Japanese characters\\textasciigrave $]$,
``similarity order'': $[$\\textasciigrave 1st most similar Japanese characters\\textasciigrave , \\textasciigrave 2nd most similar Japanese characters\\textasciigrave , \\textasciigrave 3rd most similar Japanese characters\\textasciigrave $]$,
\},
\}
"""
for l in text.split("\n")[1:-1]:
    if l!="":
        print(l + "\\\\")

In [ ]:
text = """
\{
  ``Let's'': \{
    ``phonemes'': ``\\textipa{l"Ets}'',
    ``choices'': [``\\begin{CJK}{UTF8}{min}レッツ\end{CJK}'', ``\\begin{CJK}{UTF8}{min}レツ\end{CJK}'', ``\\begin{CJK}{UTF8}{min}レテス\end{CJK}''],
    ``similarity order'': [``\\begin{CJK}{UTF8}{min}レッツ\end{CJK}'', ``\\begin{CJK}{UTF8}{min}レツ\end{CJK}'', ``\\begin{CJK}{UTF8}{min}レテス\end{CJK}'']
  \},
  ``go'': \{
    ``phonemes'': ``\\textipa{g"oU}'',
    ``choices'': [``\\begin{CJK}{UTF8}{min}ゴー\end{CJK}'', ``\\begin{CJK}{UTF8}{min}ゴウ\end{CJK}'', ``\\begin{CJK}{UTF8}{min}ゴ\end{CJK}''],
    ``similarity order'': [``\\begin{CJK}{UTF8}{min}ゴー\end{CJK}'', ``\\begin{CJK}{UTF8}{min}ゴウ\end{CJK}'', ``\\begin{CJK}{UTF8}{min}ゴ\end{CJK}'']
  \}
\}
"""
for l in text.split("\n")[1:-1]:
    if l!="":
        print(l + "\\\\")